In [2]:
from flask import Flask, render_template, request, redirect, url_for, jsonify,Blueprint
from datetime import datetime
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
import os

# 학습시킨 binary classification model 불러오기 (출력층을 sigmoid 로 설정했기에, predict 하면 아웃풋이 0~1 로 나옴)
model = tf.keras.models.load_model('tekken/static/model/model.h5',compile=False) 


2022-01-16 07:02:18.907642: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [36]:
ab = [{"name":"Paul Phoenix","Rank":1,"Winrate":49.84,"Akuma":52,"Alisa":50,"Anna":48,"ArmorKing":49,"Asuka":52,"Bob":48,"Bryan":51,"Claudio":48,"DevilJin":51,"Dragunov":51,"Eddy":50,"Eliza":51,"Fahkumram":46,"Feng":50,"Ganryu":45,"Geese":49,"Gigas":48,"Heihachi":50,"Hwoarang":50,"Jack-7":49,"Jin":54,"Josie":51,"Julia":49,"Katarina":50,"Kazumi":53,"Kazuya":52,"King":51,"Kuma":46,"Kunimitsu":50,"Lars":48,"Law":54,"Lee":52,"Lei":52,"Leo":50,"Leroy":47,"Lidia":45,"Lili":51,"LuckyChloe":51,"Marduk":46,"MasterRaven":52,"Miguel":51,"Negan":48,"Nina":51,"Noctis":50,"Panda":47,"Paul":0,"Shaheen":49,"Steve":53,"XiaoYu":53,"Yoshimitsu":55,"Zafina":45,"Omega":2.18,"youtube":"https://youtu.be/yvuHygYZlyY","stlye":"Paul은 사소한 단점과 뛰어난 장점만 있는 환상적인 캐릭터입니다. 그의 다재다능한 툴킷은 Paul이 자신이 처한 상황에 관계없이 좋은 사람이 되도록 이끕니다.수비에서 Paul은 한 번의 빗나간 히트가 치명적인 콤보로 이어질 수 있기 때문에 그에게 접근하는 것을 상당한 위험에 빠뜨리는 가장 위협적인 킵 아웃 움직임을 가지고 있습니다.","story":"그의 이름에 걸맞게 Paul Phoenix는 명성과 무술에 대한 불타는 열정을 가진 사람입니다. 외계인조차도 그가 전 우주에서 최고의 파이터라는 것을 증명하는 것을 막을 수 없습니다."},
{"name":"Kazuya","Rank":2,"Winrate":47.42, "Akuma":52,"Alisa":46,"Anna":44,"ArmorKing":49,"Asuka":48,"Bob":47,"Bryan":48,"Claudio":46,"DevilJin":50,"Dragunov":51,"Eddy":45,"Eliza":49,"Fahkumram":47,"Feng":47,"Ganryu":44,"Geese":45,"Gigas":46,"Heihachi":48,"Hwoarang":50,"Jack-7":45,"Jin":51,"Josie":49,"Julia":46,"Katarina":49,"Kazumi":51,"Kazuya":0,"King":50,"Kuma":43,"Kunimitsu":46,"Lars":47,"Law":51,"Lee":47,"Lei":48,"Leo":48,"Leroy":45,"Lidia":45,"Lili":48,"LuckyChloe":48,"Marduk":44,"MasterRaven":45,"Miguel":48,"Negan":47,"Nina":47,"Noctis":50,"Panda":44,"Paul":48,"Shaheen":48,"Steve":50,"XiaoYu":51,"Yoshimitsu":51,"Zafina":42,"Omega":3.47,"youtube":"https://youtu.be/OtNSLyntpNU","stlye":"단순한 캐릭터인 카즈야는 이 시리즈에서 조작이 쉬운 캐릭터입니다. 그는 강력한 단일 공격에 중점을 둔 이동 세트를 가지고 있지만 많은 피해를 줄 수 있습니다.","story":"Mishima Kazuya는 Mishima Zaibatsu 다국적 기업 CEO인 Heihachi Mishima의 아들입니다. 어렸을 때 생존 기술을 훈련시키기 위해 그를 벼랑에서 내던진 아버지와 다소 모호한 관계이다."},
{"name":"Bryan","Rank":3,"Winrate":49.00,"Akuma":50,"Alisa":48,"Anna":46,"ArmorKing":50,"Asuka":53,"Bob":48,"Bryan":0,"Claudio":47,"DevilJin":50,"Dragunov":52,"Eddy":47,"Eliza":48,"Fahkumram":49,"Feng":49,"Ganryu":46,"Geese":48,"Gigas":47,"Heihachi":49,"Hwoarang":50,"Jack-7":48,"Jin":54,"Josie":51,"Julia":48,"Katarina":49,"Kazumi":51,"Kazuya":52,"King":51,"Kuma":46,"Kunimitsu":48,"Lars":49,"Law":51,"Lee":50,"Lei":47,"Leo":51,"Leroy":48,"Lidia":49,"Lili":48,"LuckyChloe":46,"Marduk":47,"MasterRaven":49,"Miguel":50,"Negan":48,"Nina":48,"Noctis":51,"Panda":46,"Paul":49,"Shaheen":49,"Steve":52,"XiaoYu":49,"Yoshimitsu":53,"Zafina":44,"Omega":3.21,"youtube":"https://youtu.be/PSvyG5foCgE","stlye":"Bryan의 장점은 상대를 제치고 파괴적인 콤보로 이어지는 카운터 히트를 낚시하는 것입니다. 브라이언은 여타 캐릭터들과 다르게 커맨드가 적다.","story":"전직 경찰관 브라이언 퓨리. 과학을 위해 인생이 꼬여버린 이 남자가 이제는 재미와 지루함을 피하기 위해 혼돈속에서 호흡하며 지구를 배회합니다."},
{"name":"Fahkumram","Rank":4,"Winrate":52.50,"Akuma":56,"Alisa":54,"Anna":48,"ArmorKing":54,"Asuka":53,"Bob":52,"Bryan":51,"Claudio":51,"DevilJin":57,"Dragunov":54,"Eddy":52,"Eliza":50,"Fahkumram":0,"Feng":53,"Ganryu":52,"Geese":49,"Gigas":48,"Heihachi":52,"Hwoarang":49,"Jack-7":52,"Jin":54,"Josie":50,"Julia":53,"Katarina":51,"Kazumi":56,"Kazuya":53,"King":54,"Kuma":54,"Kunimitsu":47,"Lars":54,"Law":53,"Lee":54,"Lei":50,"Leo":52,"Leroy":52,"Lidia":47,"Lili":51,"LuckyChloe":51,"Marduk":52,"MasterRaven":53,"Miguel":51,"Negan":50,"Nina":48,"Noctis":54,"Panda":54,"Paul":54,"Shaheen":56,"Steve":54,"XiaoYu":52,"Yoshimitsu":55,"Zafina":49,"Omega":3.47,"youtube":"https://youtu.be/qImQXs2m7z4","stlye":"Fahkrumram은 매우 강하고 균형 잡힌 캐릭터입니다.그는 적당한 범위에서 사용할 수 있는 다양한 훌륭한 찌르기와 함께 자신감 있는 중립 게임을 가지고 있으며 다양한 킵 아웃 동작으로 공격적인 캐릭터를 쉽게 멀리할 수 있습니다.","story":"전설적인 무에타이 전사, Fahkumram은 남자의 산입니다. 부패한 군대가 그를 체포하고 가족을 납치한 후, Fahkumram은 그들을 대신해 싸워야 합니다."},
{"name":"Leroy","Rank":5,"Winrate":52.88,"Akuma":55,"Alisa":50,"Anna":50,"ArmorKing":56,"Asuka":53,"Bob":55,"Bryan":52,"Claudio":49,"DevilJin":55,"Dragunov":55,"Eddy":53,"Eliza":51,"Fahkumram":48,"Feng":51,"Ganryu":53,"Geese":50,"Gigas":51,"Heihachi":54,"Hwoarang":52,"Jack-7":53,"Jin":55,"Josie":54,"Julia":55,"Katarina":53,"Kazumi":53,"Kazuya":55,"King":55,"Kuma":53,"Kunimitsu":45,"Lars":55,"Law":53,"Lee":52,"Lei":52,"Leo":55,"Leroy":0,"Lidia":45,"Lili":53,"LuckyChloe":53,"Marduk":53,"MasterRaven":55,"Miguel":55,"Negan":52,"Nina":53,"Noctis":51,"Panda":50,"Paul":55,"Shaheen":53,"Steve":53,"XiaoYu":56,"Yoshimitsu":54,"Zafina":53,"Omega":1.67,"youtube":"https://youtu.be/cI6W_PSgyzw","stlye":"Leroy의 움직임 세트에는 빠른 타격과 펀치의 풍신이 많이 포함되어 있어 매우 효과적이고 다양한 카운터 도구를 제공합니다.","story":"Leroy는 잔뼈 굵은 상인 거물 중 하나입니다. 또한 단 하나의 목표를 가진 믿을 수 없을 정도로 숙련된 파이터이기도 합니다."},
{"name":"ArmorKing","Rank":6,"Winrate":49.66,"Akuma":54,"Alisa":48,"Anna":47,"ArmorKing":0,"Asuka":53,"Bob":49,"Bryan":50,"Claudio":47,"DevilJin":52,"Dragunov":51,"Eddy":50,"Eliza":50,"Fahkumram":46,"Feng":50,"Ganryu":46,"Geese":50,"Gigas":48,"Heihachi":49,"Hwoarang":49,"Jack-7":49,"Jin":54,"Josie":50,"Julia":48,"Katarina":50,"Kazumi":52,"Kazuya":51,"King":51,"Kuma":49,"Kunimitsu":46,"Lars":50,"Law":54,"Lee":51,"Lei":50,"Leo":51,"Leroy":44,"Lidia":44,"Lili":50,"LuckyChloe":50,"Marduk":46,"MasterRaven":50,"Miguel":50,"Negan":48,"Nina":49,"Noctis":51,"Panda":49,"Paul":51,"Shaheen":50,"Steve":53,"XiaoYu":52,"Yoshimitsu":54,"Zafina":45,"Omega":1.8,"youtube":"https://youtu.be/cI6W_PSgyzw","stlye":"그는 공격이나 쉬운 믹싱에 대한 추진력을 충분히 구축할 수 없지만 그의 툴킷과 움직임을 통해 상대를 편안한 범위에서 유지하면서 끊임없이 상대를 찌를 수 있습니다.","story":"철권 7에서 볼 수 있는 아머 킹은 오리지널 아머 킹의 동생인 아머 킹 II입니다."},
{"name":"Dragunov","Rank":7,"Winrate":48.36,"Akuma":52,"Alisa":48,"Anna":46,"ArmorKing":49,"Asuka":50,"Bob":48,"Bryan":48,"Claudio":48,"DevilJin":50,"Dragunov":0,"Eddy":47,"Eliza":48,"Fahkumram":46,"Feng":48,"Ganryu":45,"Geese":47,"Gigas":47,"Heihachi":48,"Hwoarang":48,"Jack-7":47,"Jin":52,"Josie":49,"Julia":47,"Katarina":49,"Kazumi":51,"Kazuya":49,"King":50,"Kuma":47,"Kunimitsu":47,"Lars":48,"Law":52,"Lee":49,"Lei":47,"Leo":50,"Leroy":45,"Lidia":46,"Lili":50,"LuckyChloe":47,"Marduk":47,"MasterRaven":48,"Miguel":49,"Negan":46,"Nina":48,"Noctis":53,"Panda":49,"Paul":49,"Shaheen":49,"Steve":51,"XiaoYu":49,"Yoshimitsu":52,"Zafina":43,"Omega":1.67,"youtube":"https://youtu.be/8q9mDi0lfIY","stlye":"Dragunov는 매우 공격적인 캐릭터입니다. 드라구노브의 공격 범위는 그가 가까운 거리에서 상대방을 제압하고 효과적으로 피해를 줄 수 있습니다.","story":"궁극의 군인. Dragunov는 그에게 주어진 임무가 무엇이든 레이저에 집중하고 최고의 효율성으로 수행한다."},
{"name":"DevilJin","Rank":8,"Winrate":47.5,"Akuma":51,"Alisa":48,"Anna":46,"ArmorKing":48,"Asuka":49,"Bob":47,"Bryan":50,"Claudio":47,"DevilJin":0,"Dragunov":50,"Eddy":46,"Eliza":46,"Fahkumram":43,"Feng":47,"Ganryu":46,"Geese":47,"Gigas":47,"Heihachi":48,"Hwoarang":49,"Jack-7":48,"Jin":50,"Josie":50,"Julia":47,"Katarina":49,"Kazumi":50,"Kazuya":50,"King":49,"Kuma":45,"Kunimitsu":46,"Lars":46,"Law":51,"Lee":47,"Lei":48,"Leo":47,"Leroy":45,"Lidia":45,"Lili":48,"LuckyChloe":46,"Marduk":46,"MasterRaven":45,"Miguel":48,"Negan":46,"Nina":48,"Noctis":45,"Panda":46,"Paul":49,"Shaheen":48,"Steve":49,"XiaoYu":48,"Yoshimitsu":51,"Zafina":44,"Omega":3.98,"youtube":"https://youtu.be/OVYepbR9Ul8","stlye":"","story":"악마의 유전자가 진의 몸과 마음을 장악한 결과. 데빌 진은 궁극의 힘과 파괴만을 마음에 품고 있는 마귀로서, 그의 길을 방해하거나 방해하는 사람은 자비가 없습니다"},
{"name":"Jin","Rank":9,"Winrate":46.3,"Akuma":48,"Alisa":47,"Anna":43,"ArmorKing":46,"Asuka":46,"Bob":46,"Bryan":46,"Claudio":45,"DevilJin":50,"Dragunov":48,"Eddy":43,"Eliza":46,"Fahkumram":46,"Feng":45,"Ganryu":43,"Geese":45,"Gigas":45,"Heihachi":47,"Hwoarang":48,"Jack-7":44,"Jin":0,"Josie":48,"Julia":45,"Katarina":48,"Kazumi":51,"Kazuya":49,"King":48,"Kuma":44,"Kunimitsu":46,"Lars":46,"Law":49,"Lee":47,"Lei":45,"Leo":47,"Leroy":45,"Lidia":45,"Lili":48,"LuckyChloe":46,"Marduk":43,"MasterRaven":45,"Miguel":46,"Negan":46,"Nina":46,"Noctis":49,"Panda":43,"Paul":46,"Shaheen":47,"Steve":49,"XiaoYu":46,"Yoshimitsu":50,"Zafina":45,"Omega":2.44,"youtube":"https://youtu.be/ASNT49PS1sI","stlye":"Jin은 Tekken 3 및 Tekken Tag Tournament에서 Mishima 스타일 전투기로 등장했습니다.그러나 실제로 Jin의 이동 세트는 Kazuya와 Jun Kazama의 하이브리드였으며 Jun의 이동 대부분은 Kazuya 이동의 개선된 버전과 함께 유지되었습니다.","story":"진 카자마는 철권 시리즈의 여러 중심 캐릭터 중 하나입니다. 그는 자신의 정신을 장악하려는 악마의 시도와 세계 지배와 관련된 Mishima Zaibatsu Corporation(및 가족)과 싸우고 있습니다."},
{"name":"King","Rank":10,"Winrate":48.54,"Akuma":52,"Alisa":47,"Anna":46,"ArmorKing":49,"Asuka":52,"Bob":47,"Bryan":49,"Claudio":47,"DevilJin":51,"Dragunov":50,"Eddy":46,"Eliza":48,"Fahkumram":46,"Feng":49,"Ganryu":46,"Geese":48,"Gigas":47,"Heihachi":49,"Hwoarang":51,"Jack-7":48,"Jin":52,"Josie":50,"Julia":47,"Katarina":48,"Kazumi":52,"Kazuya":50,"King":0,"Kuma":46,"Kunimitsu":45,"Lars":48,"Law":52,"Lee":50,"Lei":48,"Leo":50,"Leroy":45,"Lidia":48,"Lili":48,"LuckyChloe":46,"Marduk":46,"MasterRaven":49,"Miguel":49,"Negan":49,"Nina":49,"Noctis":50,"Panda":47,"Paul":49,"Shaheen":50,"Steve":52,"XiaoYu":49,"Yoshimitsu":52,"Zafina":44,"Omega":2.31,"youtube":"https://youtu.be/8Ssl0nluge4","stlye":" 대중적인 믿음과 달리 King은 노골적인 공격적인 캐릭터보다 방어적인 캐릭터에 가깝습니다. 그의 제한된 범위와 높은 위험 때문에 플레이어는 King을 신중하게 사용해야 합니다.","story":"King은 모든 철권 게임에 등장하는 모습과 존재감 덕분에 인기가 많습니다. 시리즈에는 두 명의 왕이 있습니다. 둘 다 재규어 마스크를 쓰고 있고 둘 다 멕시코 출신입니다."},
{"name":"Hwaorang","Rank":11,"Winrate":49.14,"Akuma":49,"Alisa":47,"Anna":46,"ArmorKing":51,"Asuka":49,"Bob":48,"Bryan":50,"Claudio":47,"DevilJin":51,"Dragunov":52,"Eddy":46,"Eliza":46,"Fahkumram":51,"Feng":48,"Ganryu":46,"Geese":47,"Gigas":48,"Heihachi":49,"Hwoarang":0,"Jack-7":49,"Jin":52,"Josie":50,"Julia":48,"Katarina":47,"Kazumi":52,"Kazuya":50,"King":49,"Kuma":49,"Kunimitsu":48,"Lars":49,"Law":51,"Lee":51,"Lei":50,"Leo":52,"Leroy":48,"Lidia":45,"Lili":50,"LuckyChloe":47,"Marduk":49,"MasterRaven":50,"Miguel":51,"Negan":48,"Nina":48,"Noctis":51,"Panda":49,"Paul":50,"Shaheen":50,"Steve":50,"XiaoYu":51,"Yoshimitsu":55,"Zafina":46,"Omega":3.08,"youtube":"https://youtu.be/jxaZb3dcrqE","stlye":" Hwoarang은 매우 공격적인 플레이 스타일을 전문으로 하며 그의 많은 움직임은 블록 위에서도 그를 유리한 상태로 만듭니다.수비력이 좋지 않아 지속적으로 상대를 압박하는 데 적격이다.","story":"화랑은 태권도 사범 백두산의 제자다. 미시마 헤이하치가 풀어준 오우거가 스승을 죽였기 때문에 제3차 King of Iron Fist 토너먼트에 참가했다."},
{"name":"Negan","Rank":12,"Winrate":51.58,"Akuma":51,"Alisa":50,"Anna":48,"ArmorKing":52,"Asuka":56,"Bob":54,"Bryan":52,"Claudio":51,"DevilJin":54,"Dragunov":54,"Eddy":52,"Eliza":51,"Fahkumram":50,"Feng":52,"Ganryu":48,"Geese":46,"Gigas":46,"Heihachi":53,"Hwoarang":52,"Jack-7":50,"Jin":54,"Josie":52,"Julia":51,"Katarina":52,"Kazumi":55,"Kazuya":53,"King":51,"Kuma":51,"Kunimitsu":51,"Lars":52,"Law":54,"Lee":53,"Lei":53,"Leo":53,"Leroy":48,"Lidia":45,"Lili":54,"LuckyChloe":51,"Marduk":47,"MasterRaven":54,"Miguel":53,"Negan":0,"Nina":51,"Noctis":53,"Panda":50,"Paul":52,"Shaheen":53,"Steve":53,"XiaoYu":56,"Yoshimitsu":55,"Zafina":50,"Omega":1.54,"youtube":"https://youtu.be/gqZlu2EsjJU","stlye":"  솔직한 싸움꾼인 Negan은 적과 가까이 있을 때 가장 기분이 좋습니다. 블록에서 그를 프레임 이점으로 남겨 둘 수 있는 일련의 훌륭한 찌르기,강한 자세 움직임, 안전한 일반 히트 런처를 통해 Negan은 효율적으로 상대를 제압할 수 있습니다.그의 가차없는 공격은 상대가 반격을 시도하고 실수를 저지르거나 Negan의 무서운 파워 크러쉬 중 하나를 공격할 때까지 계속됩니다.","story":" 생존자 그룹의 무자비한 지도자로서 Negan은 필요한 모든 수단을 통해 그들의 번영과 생존을 보장할 것입니다."},
{"name":"Heihachi","Rank":13,"Winrate":48.96,"Akuma":51,"Alisa":47,"Anna":46,"ArmorKing":51,"Asuka":50,"Bob":47,"Bryan":51,"Claudio":47,"DevilJin":52,"Dragunov":52,"Eddy":45,"Eliza":47,"Fahkumram":48,"Feng":49,"Ganryu":47,"Geese":50,"Gigas":49,"Heihachi":0,"Hwoarang":51,"Jack-7":49,"Jin":53,"Josie":52,"Julia":48,"Katarina":47,"Kazumi":52,"Kazuya":52,"King":51,"Kuma":46,"Kunimitsu":49,"Lars":48,"Law":50,"Lee":49,"Lei":49,"Leo":50,"Leroy":46,"Lidia":46,"Lili":49,"LuckyChloe":48,"Marduk":47,"MasterRaven":48,"Miguel":49,"Negan":47,"Nina":48,"Noctis":48,"Panda":46,"Paul":50,"Shaheen":52,"Steve":51,"XiaoYu":51,"Yoshimitsu":53,"Zafina":44,"Omega":1.8,"youtube":"https://youtu.be/img9BH_iHzs","stlye":"Heihachi는 그의 파괴적이고 압도적인 움직임으로 인해 공격적인 플레이어에게 최고의 옵션 중 하나입니다.그의 최선의 움직임의 대부분은 위험이 거의 또는 전혀 없으며 그의 손상 가능성은 매우 높습니다.","story":"미시마 헤이하치는 철권 시리즈의 주인공이자 악당이다. King of Iron Fist Tournament의 주최자이자 Mishima Corporation의 소유주이자 무한한 권력에 대한 집착은 그의 평생 노력의 일부일 뿐입니다."},
{"name":"SteveFox","Rank":14,"Winrate":46.46,"Akuma":46,"Alisa":46,"Anna":43,"ArmorKing":47,"Asuka":44,"Bob":47,"Bryan":48,"Claudio":43,"DevilJin":51,"Dragunov":49,"Eddy":41,"Eliza":44,"Fahkumram":46,"Feng":45,"Ganryu":44,"Geese":44,"Gigas":45,"Heihachi":49,"Hwoarang":50,"Jack-7":45,"Jin":51,"Josie":51,"Julia":46,"Katarina":45,"Kazumi":51,"Kazuya":50,"King":48,"Kuma":44,"Kunimitsu":44,"Lars":45,"Law":47,"Lee":48,"Lei":44,"Leo":49,"Leroy":47,"Lidia":44,"Lili":47,"LuckyChloe":43,"Marduk":46,"MasterRaven":47,"Miguel":49,"Negan":47,"Nina":46,"Noctis":47,"Panda":44,"Paul":47,"Shaheen":50,"Steve":0,"XiaoYu":47,"Yoshimitsu":50,"Zafina":43,"Omega":4.36,"youtube":"https://youtu.be/9mzv0cVNcgA","stlye":"최고의 캐릭터 중 하나인 Steve는 어떤 상황에서도 사용할 수 있는 견고하고 다재다능한 툴킷을 보유하고 있습니다. 그러나 그의 독특한 강점과 약점은 그의 플레이 스타일을 균형 잡힌 다른 캐릭터와 차별화합니다.다양한 움직임 옵션, 자세, 찌르기를 통해 Steve는 효율적으로 상대를 제압하고 일련의 펀치로 상대를 붙잡고 상대가 열릴 때까지 가차 없이 압박할 수 있습니다.","story":" 난관에 봉착한 강력한 복서. 스티브 폭스는 과거의 비밀을 밝히고 아무도 비슷한 운명을 겪지 않도록 하기 위해 싸웁니다."},
{"name":"Kazumi","Rank":15,"Winrate":47.36,"Akuma":45,"Alisa":45,"Anna":43,"ArmorKing":48,"Asuka":50,"Bob":48,"Bryan":49,"Claudio":44,"DevilJin":50,"Dragunov":49,"Eddy":46,"Eliza":46,"Fahkumram":44,"Feng":49,"Ganryu":45,"Geese":44,"Gigas":46,"Heihachi":48,"Hwoarang":48,"Jack-7":46,"Jin":49,"Josie":50,"Julia":47,"Katarina":48,"Kazumi":0,"Kazuya":49,"King":48,"Kuma":45,"Kunimitsu":45,"Lars":48,"Law":49,"Lee":49,"Lei":49,"Leo":49,"Leroy":47,"Lidia":44,"Lili":49,"LuckyChloe":48,"Marduk":46,"MasterRaven":49,"Miguel":48,"Negan":45,"Nina":47,"Noctis":47,"Panda":45,"Paul":47,"Shaheen":50,"Steve":49,"XiaoYu":51,"Yoshimitsu":53,"Zafina":46,"Omega":1.93,"youtube":"https://youtu.be/JJzxIMTF_u8","stlye":"Kazumi는 Heihachi와 Kazuya의 움직임을 조합하여 악의적인 펀치와 핸드 찹을 사용하여 상대를 물리칩니다.그녀의 독특한 능력은 애완용 호랑이를 소환하는 것인데, 이는 콤보의 일부로 또는 단독으로 수행할 수 있습니다. 그것은 같은 장면에서 카즈미가 호랑이와 함께 어퍼컷을 한 후에도 볼 수 있습니다.","story":"Heihachi Mishima의 사망한 아내인 Kazumi Mishima는 Heihachi의 집권과 세계적인 불안정을 막으려 했습니다. 아아, 그녀의 시도는 실패했고 대신 시리즈의 사건을 촉발시키는 데 도움이 될 것입니다."},
{"name":"Law","Rank":16,"Winrate":48.02,"Akuma":50,"Alisa":46,"Anna":46,"ArmorKing":46,"Asuka":49,"Bob":47,"Bryan":49,"Claudio":46,"DevilJin":49,"Dragunov":48,"Eddy":50,"Eliza":48,"Fahkumram":47,"Feng":48,"Ganryu":43,"Geese":47,"Gigas":47,"Heihachi":50,"Hwoarang":49,"Jack-7":48,"Jin":51,"Josie":48,"Julia":47,"Katarina":48,"Kazumi":51,"Kazuya":49,"King":48,"Kuma":48,"Kunimitsu":46,"Lars":48,"Law":0,"Lee":49,"Lei":48,"Leo":50,"Leroy":47,"Lidia":47,"Lili":49,"LuckyChloe":45,"Marduk":47,"MasterRaven":50,"Miguel":49,"Negan":46,"Nina":49,"Noctis":48,"Panda":47,"Paul":46,"Shaheen":48,"Steve":53,"XiaoYu":50,"Yoshimitsu":52,"Zafina":45,"Omega":2.18,"youtube":"https://youtu.be/a0Nof2bMwak","stlye":"그러나 그의 실제 동작 세트는 이소룡의 동작에서 크게 영감을 받았으므로 Law의 실제 전투 스타일은 절권도(무술가 이소룡이 창시한 하이브리드 무술 시스템 및 삶의 철학)를 기반으로 한다고 가정하는 것이 안전합니다.","story":"마샬 로(Marshall Law)는 일본의 무술가입니다. 그는 자신의 도장을 꿈꾸었고 그것을 실현했습니다. 동시에 그는 여러 King of the Iron Fist 토너먼트에 참가했으며,그는 상대를 물리칠 수 있었지만 전체적으로는 패배했습니다. 그의 장점은 강한 단타가 아니라 수에 있다."},
{"name":"Lars","Rank":17,"Winrate":50.16,"Akuma":51,"Alisa":48,"Anna":47,"ArmorKing":50,"Asuka":54,"Bob":49,"Bryan":51,"Claudio":46,"DevilJin":54,"Dragunov":52,"Eddy":51,"Eliza":49,"Fahkumram":46,"Feng":51,"Ganryu":48,"Geese":47,"Gigas":49,"Heihachi":52,"Hwoarang":51,"Jack-7":50,"Jin":54,"Josie":51,"Julia":51,"Katarina":49,"Kazumi":52,"Kazuya":53,"King":52,"Kuma":48,"Kunimitsu":47,"Lars":0,"Law":52,"Lee":52,"Lei":49,"Leo":52,"Leroy":45,"Lidia":45,"Lili":50,"LuckyChloe":49,"Marduk":49,"MasterRaven":51,"Miguel":51,"Negan":48,"Nina":50,"Noctis":51,"Panda":48,"Paul":52,"Shaheen":52,"Steve":55,"XiaoYu":53,"Yoshimitsu":55,"Zafina":47,"Omega":2.57,"youtube":"https://youtu.be/gK0nkLOBGXQ","stlye":"Lars는 시리즈에서 더 유연한 전투기 중 하나이며 가장 배우기 쉽고 사용하기 쉬운 전투기 중 하나입니다. Lars의 전투 스타일은 실제 무술을 기반으로하지 않으며 완전히 구성되어 있습니다.철권 7에서 그의 전투 스타일은 철권군 무술입니다.","story":"Lars Alexandersson은 아버지와 형제의 다국적 기업인 Mishima Zaibatsu가 촉발한 무시무시한 전쟁을 멈추기로 결정한 Heihachi Mishima의 사생아입니다. 철권 분대의 일원이다."},
{"name":"Claudio","Rank":18,"Winrate":52.34,"Akuma":52,"Alisa":52,"Anna":50,"ArmorKing":53,"Asuka":55,"Bob":51,"Bryan":53,"Claudio":0,"DevilJin":53,"Dragunov":52,"Eddy":51,"Eliza":54,"Fahkumram":49,"Feng":52,"Ganryu":46,"Geese":51,"Gigas":52,"Heihachi":53,"Hwoarang":53,"Jack-7":53,"Jin":55,"Josie":53,"Julia":51,"Katarina":52,"Kazumi":56,"Kazuya":54,"King":53,"Kuma":51,"Kunimitsu":48,"Lars":54,"Law":54,"Lee":55,"Lei":53,"Leo":54,"Leroy":51,"Lidia":45,"Lili":53,"LuckyChloe":52,"Marduk":50,"MasterRaven":52,"Miguel":51,"Negan":49,"Nina":53,"Noctis":53,"Panda":52,"Paul":52,"Shaheen":52,"Steve":57,"XiaoYu":57,"Yoshimitsu":59,"Zafina":51,"Omega":1.8,"youtube":"https://youtu.be/h9gXoB8Df44","stlye":"Claudio는 57개의 움직임으로 Tekken 7 캐릭터 중에서 가장 적은 움직임을 가지고 있습니다. 다양성이 부족함에도 불구하고 그의 핵심 툴킷은 매우 강력하며 좋은 접근 방식과 배제 옵션을 제공합니다.그는 또한 매우 강력하고 안전한 움직임에 접근할 수 있는 독특한 파워업 메커니즘을 가지고 있습니다. 일반적으로 클라우디오는 상대방을 벽으로 밀어붙일 때 가장 강합니다.","story":"세계에서 초자연적인 영향을 추방하는 임무를 맡은 비밀 결사, 시리우스의 궁수의 뛰어난 퇴마사 클라우디오 세라피노는 헤이하치와 나머지 미시마 재벌들이 숨기고 있는 비밀을 찾기 위해 나선다."},
{"name":"Josie","Rank":19,"Winrate":48.5,"Akuma":49,"Alisa":47,"Anna":44,"ArmorKing":50,"Asuka":50,"Bob":49,"Bryan":49,"Claudio":47,"DevilJin":50,"Dragunov":51,"Eddy":49,"Eliza":47,"Fahkumram":50,"Feng":48,"Ganryu":46,"Geese":44,"Gigas":47,"Heihachi":48,"Hwoarang":50,"Jack-7":49,"Jin":52,"Josie":0,"Julia":46,"Katarina":48,"Kazumi":50,"Kazuya":51,"King":50,"Kuma":48,"Kunimitsu":46,"Lars":49,"Law":52,"Lee":48,"Lei":49,"Leo":51,"Leroy":46,"Lidia":46,"Lili":49,"LuckyChloe":47,"Marduk":49,"MasterRaven":48,"Miguel":49,"Negan":48,"Nina":47,"Noctis":51,"Panda":46,"Paul":49,"Shaheen":49,"Steve":49,"XiaoYu":51,"Yoshimitsu":52,"Zafina":45,"Omega":1.67,"youtube":"https://youtu.be/6G_CoB5N79Y","stlye":"Josie는 매우 민첩하고 빠른 캐릭터이며 전반적인 피해량이 높기 때문에 특히 처음 플레이하는 플레이어에게 선택하고 플레이할 수 있는 매우 간단한 캐릭터입니다.한 가지 명심해야 할 점은 시리즈의 대다수 여성 캐릭터와 달리 그녀는 백 핸드스프링을 수행할 수 없다는 것입니다.","story":"Josie Rizal은 흐느끼는 것이 빠르지만 해야 할 때는 진정한 파이터입니다. 철권 7에 등장하는 새로운 캐릭터 중 하나. 킥복싱 신동이지만 가끔은 그녀도 자신의 승리가 믿기지 않을 때가 있다."},
{"name":"Noctis","Rank":20,"Winrate":50.12,"Akuma":55,"Alisa":49,"Anna":46,"ArmorKing":49,"Asuka":56,"Bob":50,"Bryan":49,"Claudio":47,"DevilJin":55,"Dragunov":47,"Eddy":55,"Eliza":52,"Fahkumram":46,"Feng":51,"Ganryu":46,"Geese":48,"Gigas":48,"Heihachi":52,"Hwoarang":49,"Jack-7":54,"Jin":51,"Josie":49,"Julia":47,"Katarina":50,"Kazumi":53,"Kazuya":50,"King":50,"Kuma":50,"Kunimitsu":46,"Lars":49,"Law":52,"Lee":53,"Lei":52,"Leo":49,"Leroy":49,"Lidia":43,"Lili":49,"LuckyChloe":52,"Marduk":46,"MasterRaven":54,"Miguel":50,"Negan":47,"Nina":49,"Noctis":0,"Panda":51,"Paul":50,"Shaheen":51,"Steve":53,"XiaoYu":54,"Yoshimitsu":56,"Zafina":47,"Omega":2.18,"youtube":"https://youtu.be/dNACrBDc9Lk","stlye":"Final Fantasy XV를 플레이한 모든 사람은 원래 게임에서 대부분 유지되었기 때문에 Noctis의 움직임을 많이 인식할 것입니다.전반적으로 Noctis는 상대를 멀리하고 블록과 휙으로 처벌하는 다양한 도구를 갖춘 매우 단순한 방어 캐릭터입니다.","story":"Noctis Lucis Caelum은 상징적인 Final Fantasy 프랜차이즈의 캐릭터입니다. 정확히는 파이널 판타지 XV.자신의 감정을 털어놓기 위해 고군분투하는 불안에 휩싸인 청년은 조국을 구해야 하는 극도의 스트레스 상황에 뛰어듭니다. 적어도 그는 친구들과 잠시 동안 낚시를 할 수 있습니다."},
{"name":"Miguel","Rank":21,"Winrate":49.30,"Akuma":53,"Alisa":49,"Anna":47,"ArmorKing":50,"Asuka":49,"Bob":48,"Bryan":50,"Claudio":49,"DevilJin":52,"Dragunov":51,"Eddy":47,"Eliza":48,"Fahkumram":49,"Feng":49,"Ganryu":46,"Geese":46,"Gigas":48,"Heihachi":51,"Hwoarang":49,"Jack-7":51,"Jin":54,"Josie":51,"Julia":50,"Katarina":48,"Kazumi":52,"Kazuya":52,"King":51,"Kuma":50,"Kunimitsu":47,"Lars":49,"Law":51,"Lee":50,"Lei":49,"Leo":52,"Leroy":45,"Lidia":42,"Lili":50,"LuckyChloe":46,"Marduk":49,"MasterRaven":50,"Miguel":0,"Negan":47,"Nina":49,"Noctis":50,"Panda":50,"Paul":49,"Shaheen":51,"Steve":51,"XiaoYu":50,"Yoshimitsu":54,"Zafina":47,"Omega":1.67,"youtube":"https://youtu.be/j78LPFt9fiY","stlye":"미구엘은 자신의 성격에 맞게 돌진하고 공격적인 캐릭터입니다.그의 툴킷은 Miguel을 근거리에서 매우 위협적으로 만들고 상대방의 건강을 깎아내리는 단단한 찌르기와 믹스업과 상대방이 반격을 방해하거나 시작하려고 하지 않도록 유도하는 카운터 히트 런처를 제공합니다. 근거리에서의 놀라운 기량을 만회하기 위해 Miguel은 중거리 및 장거리에서 고군분투합니다. 그의 움직임 대부분은 상대를 막을 수 있는 범위가 부족하고 그의 접근 도구는 상당한 위험을 수반하므로 심각한 처벌을 받을 수 있습니다.","story":"Mishima Zaibatsu의 손에 사랑하는 여동생이 사망한 후 Miguel Caballero Rojo는 복수의 길을 떠납니다.맹렬한 힘과 야만적인 전투 본능으로 미구엘은 진 카자마를 향해 갑니다."},
{"name":"Shaheen","Rank":22,"Winrate":48.80,"Akuma":51,"Alisa":47,"Anna":48,"ArmorKing":50,"Asuka":51,"Bob":48,"Bryan":51,"Claudio":48,"DevilJin":52,"Dragunov":51,"Eddy":45,"Eliza":46,"Fahkumram":44,"Feng":50,"Ganryu":45,"Geese":47,"Gigas":48,"Heihachi":48,"Hwoarang":50,"Jack-7":47,"Jin":53,"Josie":51,"Julia":49,"Katarina":50,"Kazumi":50,"Kazuya":52,"King":50,"Kuma":46,"Kunimitsu":45,"Lars":48,"Law":52,"Lee":48,"Lei":49,"Leo":50,"Leroy":47,"Lidia":44,"Lili":51,"LuckyChloe":47,"Marduk":47,"MasterRaven":49,"Miguel":49,"Negan":47,"Nina":49,"Noctis":49,"Panda":47,"Paul":51,"Shaheen":0,"Steve":50,"XiaoYu":53,"Yoshimitsu":54,"Zafina":46,"Omega":1.8,"youtube":"https://youtu.be/lXFmXWJ3WTg","stlye":"모든 거래의 잭인 Shaheen은 눈에 띄는 약점이 없는 다재다능하고 직선적인 캐릭터입니다. 그의 툴킷은 광범위하지는 않지만 철권 캐릭터에서 찾을 수 있는 모든 것을 갖추고 있습니다.적절한 움직임, 좋은 찌르기, 견고한 믹스업 및 필요한 모든 기본 동작. 이러한 점들 중 어느 것도 그 자체로 탁월하지는 않지만, 그들의 집합적인 힘은 Shaheen을 실행 가능하고 경쟁력 있는 성격으로 만듭니다.","story":"중동에서 온 CQC의 자비로운 마스터. 친구가 불가사의한 상황에서 사망한 후, Shaheen은 범인을 찾기 위한 탐구에 착수합니다."},
{"name":"Marduk","Rank":23,"Winrate":51.68,"Akuma":54,"Alisa":49,"Anna":46,"ArmorKing":54,"Asuka":54,"Bob":51,"Bryan":53,"Claudio":50,"DevilJin":54,"Dragunov":53,"Eddy":54,"Eliza":51,"Fahkumram":48,"Feng":53,"Ganryu":48,"Geese":51,"Gigas":53,"Heihachi":53,"Hwoarang":51,"Jack-7":52,"Jin":57,"Josie":51,"Julia":52,"Katarina":52,"Kazumi":54,"Kazuya":56,"King":54,"Kuma":51,"Kunimitsu":47,"Lars":51,"Law":53,"Lee":53,"Lei":50,"Leo":53,"Leroy":47,"Lidia":44,"Lili":51,"LuckyChloe":49,"Marduk":0,"MasterRaven":53,"Miguel":51,"Negan":53,"Nina":49,"Noctis":54,"Panda":49,"Paul":54,"Shaheen":53,"Steve":54,"XiaoYu":54,"Yoshimitsu":54,"Zafina":49,"Omega":1.93,"youtube":"https://youtu.be/CeFJv7CY82s","stlye":"Craig는 Vale Tudo를 사용하여 싸웁니다. Vale Tudo는 레슬링에서 유도, 삼보, 무에타이에 이르기까지 다양한 전투 스타일을 포괄하는 규칙이 거의 없는 전투 스포츠입니다.말 그대로 무엇이든 간다는 의미의 Vale Tudo라는 이름에 걸맞게 Marduk은 특정한 전투 스타일이 없는 것 같고 Miguel처럼 Brawling의 스타일에 더 익숙합니다.","story":"Craig Marduk은 Vale Tudo 전투에서 다양한 기술을 사용합니다. 이 철권 파이터는 크고 느리지만 공격에 성공할 때 희생자는 많은 건강을 잃습니다.Marduk은 공중에서도 효과적인 잡기를 수행합니다. 성격은 폭력적이며 감정적으로 표현한다."},
{"name":"Asuka","Rank":24,"Winrate":47.98,"Akuma":48,"Alisa":43,"Anna":47,"ArmorKing":47,"Asuka":0,"Bob":48,"Bryan":47,"Claudio":45,"DevilJin":51,"Dragunov":50,"Eddy":46,"Eliza":47,"Fahkumram":47,"Feng":48,"Ganryu":46,"Geese":47,"Gigas":46,"Heihachi":50,"Hwoarang":51,"Jack-7":47,"Jin":54,"Josie":50,"Julia":46,"Katarina":48,"Kazumi":50,"Kazuya":52,"King":48,"Kuma":45,"Kunimitsu":44,"Lars":46,"Law":51,"Lee":50,"Lei":51,"Leo":49,"Leroy":47,"Lidia":47,"Lili":47,"LuckyChloe":46,"Marduk":46,"MasterRaven":48,"Miguel":51,"Negan":44,"Nina":49,"Noctis":44,"Panda":46,"Paul":48,"Shaheen":49,"Steve":56,"XiaoYu":50,"Yoshimitsu":51,"Zafina":45,"Omega":2.05,"youtube":"https://youtu.be/RyWanj6xjc8","stlye":"Asuka는 상대방을 공격범위 안으로 유인하고 강력한 킵 아웃 동작과 Whiff Punishment를 활용할 때 빛을 발합니다.","story":"아스카는 어렸을 때부터 아버지로부터 카자마 무술을 배웠습니다. 그녀는 정의와 공정성의 가치를 가지고 자랐습니다.집에 돌아왔을 때, 그녀는 아버지의 많은 제자들이 구타를 당하는 것을 발견했습니다. 그들은 도장에 가서 그들과 Asuka의 아버지를 공격하여 그를 입원시킨 중국인에 대해 나중에 Feng Wei로 밝혀졌습니다."},
{"name":"Julia","Rank":25,"Winrate":50.48,"Akuma":52,"Alisa":51,"Anna":47,"ArmorKing":52,"Asuka":54,"Bob":50,"Bryan":52,"Claudio":49,"DevilJin":53,"Dragunov":53,"Eddy":50,"Eliza":50,"Fahkumram":47,"Feng":50,"Ganryu":46,"Geese":48,"Gigas":49,"Heihachi":52,"Hwoarang":52,"Jack-7":48,"Jin":55,"Josie":54,"Julia":0,"Katarina":49,"Kazumi":53,"Kazuya":54,"King":53,"Kuma":47,"Kunimitsu":47,"Lars":49,"Law":53,"Lee":53,"Lei":52,"Leo":51,"Leroy":45,"Lidia":44,"Lili":51,"LuckyChloe":50,"Marduk":48,"MasterRaven":50,"Miguel":50,"Negan":49,"Nina":50,"Noctis":53,"Panda":46,"Paul":51,"Shaheen":51,"Steve":54,"XiaoYu":54,"Yoshimitsu":54,"Zafina":47,"Omega":1.93,"youtube":"https://youtu.be/k8t63kqF4Kk","stlye":"양어머니 미셸 밑에서 자란 줄리아는 외모와 행동이 미셸과 거의 흡사한 전투 스타일을 갖고 있다.Julia의 발차기와 빠른 공격에는 여러 가지 변형이 있으며, 그 중 많은 부분이 다른 공격과 연결될 수 있으므로 동일한 동작을 여러 번 변형할 수 있습니다.","story":"Michelle Chang은 그녀의 의붓딸 Julia에게 매우 효과적인 공격을 가르쳤습니다. Julia는 일부 레슬링 기술과 중국 무술을 사용합니다. 그녀는 매우 빠르며 콤보로 쉽게 연결될 수 있는 빠른 히트로 적에게 피해를 줍니다."},
{"name":"Feng","Rank":26,"Winrate":49.76,"Akuma":50,"Alisa":44,"Anna":49,"ArmorKing":50,"Asuka":52,"Bob":47,"Bryan":51,"Claudio":48,"DevilJin":53,"Dragunov":52,"Eddy":49,"Eliza":46,"Fahkumram":47,"Feng":0,"Ganryu":49,"Geese":48,"Gigas":46,"Heihachi":51,"Hwoarang":52,"Jack-7":48,"Jin":55,"Josie":52,"Julia":50,"Katarina":48,"Kazumi":51,"Kazuya":53,"King":51,"Kuma":47,"Kunimitsu":48,"Lars":49,"Law":52,"Lee":51,"Lei":53,"Leo":51,"Leroy":49,"Lidia":49,"Lili":50,"LuckyChloe":48,"Marduk":47,"MasterRaven":49,"Miguel":51,"Negan":48,"Nina":49,"Noctis":49,"Panda":47,"Paul":50,"Shaheen":50,"Steve":55,"XiaoYu":54,"Yoshimitsu":52,"Zafina":49,"Omega":2.31,"youtube":"https://youtu.be/6VGD-2vXbG4","stlye":"Feng은 확실한 뺑소니 스타일의 캐릭터입니다. 그의 콤보 데미지가 약한 편에 있고 견고한 발사 움직임이 많지 않은 반면, Feng은 유연한 툴킷으로 이를 만회하고 있습니다.","story":"치명적인 켄포 마스터. Feng Wei는 인간의 힘과 무술 숙달의 절정에 도달하기로 결심했습니다. 이러한 목표를 달성하기 위해 해야 할 일은 무엇이든 Feng은 아무 생각 없이 달성할 것입니다."},
{"name":"Lili","Rank":27,"Winrate":49.88,"Akuma":51,"Alisa":48,"Anna":48,"ArmorKing":50,"Asuka":53,"Bob":47,"Bryan":52,"Claudio":47,"DevilJin":52,"Dragunov":50,"Eddy":51,"Eliza":49,"Fahkumram":49,"Feng":50,"Ganryu":45,"Geese":49,"Gigas":48,"Heihachi":51,"Hwoarang":50,"Jack-7":50,"Jin":52,"Josie":51,"Julia":49,"Katarina":49,"Kazumi":51,"Kazuya":52,"King":52,"Kuma":47,"Kunimitsu":46,"Lars":50,"Law":51,"Lee":51,"Lei":55,"Leo":52,"Leroy":47,"Lidia":48,"Lili":0,"LuckyChloe":49,"Marduk":49,"MasterRaven":52,"Miguel":50,"Negan":46,"Nina":50,"Noctis":51,"Panda":47,"Paul":49,"Shaheen":49,"Steve":53,"XiaoYu":53,"Yoshimitsu":56,"Zafina":47,"Omega":1.8,"youtube":"https://youtu.be/aB0Sw0NUfvo","stlye":"Lili의 격투 스타일은 우아하고 다재다능한 회전, 수레바퀴, 물구나무서기, 공중제비 및 피루엣과 같은 많은 춤과 같은 곡예 동작과 결합된 체조 동작이 특징입니다.","story":"에밀리의 아버지는 부유한 사업가이며 딸의 이상한 취미를 거부합니다.그녀의 우수한 교육과 완벽한 매너에도 불구하고 Lili는 매우 숙련된 파이터입니다! 그녀의 스타일은 체조, 춤, 곡예를 기반으로 합니다."},
{"name":"Lee","Rank":28,"Winrate":48.66,"Akuma":49,"Alisa":45,"Anna":46,"ArmorKing":49,"Asuka":50,"Bob":47,"Bryan":50,"Claudio":45,"DevilJin":53,"Dragunov":51,"Eddy":46,"Eliza":45,"Fahkumram":46,"Feng":49,"Ganryu":47,"Geese":46,"Gigas":48,"Heihachi":51,"Hwoarang":49,"Jack-7":47,"Jin":53,"Josie":52,"Julia":47,"Katarina":48,"Kazumi":51,"Kazuya":53,"King":50,"Kuma":47,"Kunimitsu":45,"Lars":48,"Law":51,"Lee":0,"Lei":50,"Leo":51,"Leroy":48,"Lidia":47,"Lili":49,"LuckyChloe":46,"Marduk":47,"MasterRaven":49,"Miguel":50,"Negan":47,"Nina":48,"Noctis":47,"Panda":47,"Paul":48,"Shaheen":52,"Steve":52,"XiaoYu":52,"Yoshimitsu":53,"Zafina":46,"Omega":1.8,"youtube":"https://youtu.be/rM0WB7-ekBg","stlye":"Marshall Law의 기술(이는 Bruce Lee의 움직임에서 크게 영감을 받음)의 변형이므로 Lee의 실제 전투 스타일은 Jet Kune Do를 기반으로 한다고 가정하는 것이 안전합니다.","story":"Lee는 그의 무덤까지 그를 따라갈 많은 가족 문제가 있는 기업가적인 무술가입니다. 입양한 형 카즈야의 훈련체로 받아들여졌다. 예상대로 미시마 가족의 휴가는 말할 것도 없이 팽팽하다."},
{"name":"Jack-7","Rank":29,"Winrate":50.48,"Akuma":51,"Alisa":45,"Anna":51,"ArmorKing":51,"Asuka":53,"Bob":49,"Bryan":52,"Claudio":47,"DevilJin":52,"Dragunov":53,"Eddy":49,"Eliza":49,"Fahkumram":48,"Feng":52,"Ganryu":51,"Geese":49,"Gigas":49,"Heihachi":51,"Hwoarang":51,"Jack-7":0,"Jin":56,"Josie":51,"Julia":52,"Katarina":50,"Kazumi":54,"Kazuya":55,"King":52,"Kuma":50,"Kunimitsu":45,"Lars":50,"Law":52,"Lee":53,"Lei":50,"Leo":53,"Leroy":47,"Lidia":46,"Lili":50,"LuckyChloe":48,"Marduk":48,"MasterRaven":50,"Miguel":49,"Negan":50,"Nina":49,"Noctis":46,"Panda":52,"Paul":51,"Shaheen":53,"Steve":55,"XiaoYu":53,"Yoshimitsu":54,"Zafina":47,"Omega":1.41,"youtube":"https://youtu.be/N-8X8yHxb48","stlye":"모든 Jack 로봇은 크고 강력한 전투기이기 때문에 적을 쓰러뜨리는 데 몇 번의 확실한 타격만 있으면 됩니다. 그러나 느리고 제한된 이동 세트를 예측할 수 있기 때문에 플레이어의 적절한 타이밍이 필요합니다.","story":"Jack 프로젝트는 소련의 손에 있는 초무기였습니다. 정부를 무너뜨릴 수 있는 양산형 전투기.시간이 흐르고 러시아군이 잭 프로젝트를 중단함에 따라, 그것은 서서히 이상한 로봇이 되었습니다. 철권 세계의 많은 것 중 하나."},
{"name":"Alisa","Rank":30,"Winrate":52.32,"Akuma":53,"Alisa":0,"Anna":50,"ArmorKing":52,"Asuka":57,"Bob":52,"Bryan":52,"Claudio":48,"DevilJin":52,"Dragunov":52,"Eddy":56,"Eliza":52,"Fahkumram":46,"Feng":56,"Ganryu":49,"Geese":49,"Gigas":52,"Heihachi":53,"Hwoarang":53,"Jack-7":55,"Jin":53,"Josie":53,"Julia":49,"Katarina":52,"Kazumi":55,"Kazuya":54,"King":53,"Kuma":55,"Kunimitsu":48,"Lars":52,"Law":54,"Lee":55,"Lei":56,"Leo":53,"Leroy":50,"Lidia":47,"Lili":52,"LuckyChloe":53,"Marduk":51,"MasterRaven":56,"Miguel":51,"Negan":50,"Nina":51,"Noctis":51,"Panda":52,"Paul":50,"Shaheen":53,"Steve":54,"XiaoYu":56,"Yoshimitsu":59,"Zafina":49,"Omega":2.18,"youtube":"https://youtu.be/ulP6KPvCoZI","stlye":"Alisa는 매우 민첩하고 빠른 캐릭터입니다. 그녀의 이동성과 좋은 범위는 Alisa를 현명한 위치 지정과 신중한 플레이어에게 보상하는 훌륭한 찌르기 캐릭터입니다.Alisa의 독특한 기능 중 하나는 두 팔이 전기톱으로 변하는 자세로 전환할 수 있다는 것입니다.","story":"Bosconovitch 박사는 Alisa Bosconovitch와 관련하여 참으로 놀라운 기술을 만들어냈습니다."},
{"name":"Katarina","Rank":31,"Winrate":50.78,"Akuma":53,"Alisa":48,"Anna":50,"ArmorKing":50,"Asuka":52,"Bob":51,"Bryan":51,"Claudio":48,"DevilJin":51,"Dragunov":51,"Eddy":51,"Eliza":51,"Fahkumram":49,"Feng":52,"Ganryu":48,"Geese":49,"Gigas":49,"Heihachi":53,"Hwoarang":53,"Jack-7":50,"Jin":52,"Josie":52,"Julia":51,"Katarina":0,"Kazumi":52,"Kazuya":51,"King":52,"Kuma":50,"Kunimitsu":48,"Lars":51,"Law":52,"Lee":52,"Lei":54,"Leo":53,"Leroy":47,"Lidia":47,"Lili":51,"LuckyChloe":49,"Marduk":48,"MasterRaven":53,"Miguel":52,"Negan":48,"Nina":52,"Noctis":50,"Panda":51,"Paul":50,"Shaheen":50,"Steve":55,"XiaoYu":55,"Yoshimitsu":55,"Zafina":46,"Omega":1.54,"youtube":"https://youtu.be/rhide8oNCcQ","stlye":"  Katarina는 전체 시리즈에서 사용하는 가장 간단한 캐릭터 중 하나로 잘 알려져 있습니다.그녀는 시리즈의 다른 캐릭터로 사용하기에 지나치게 복잡하지도 어렵지도 않습니다.","story":"Katarina Alves는 Savate Brazil 마스터인 철권 7에 도입된 5명의 새로운 캐릭터 중 하나입니다."},
{"name":"Kunimitsu","Rank":32,"Winrate":52.98,"Akuma":55,"Alisa":52,"Anna":48,"ArmorKing":54,"Asuka":56,"Bob":54,"Bryan":52,"Claudio":52,"DevilJin":54,"Dragunov":53,"Eddy":56,"Eliza":57,"Fahkumram":53,"Feng":52,"Ganryu":51,"Geese":51,"Gigas":49,"Heihachi":51,"Hwoarang":52,"Jack-7":55,"Jin":54,"Josie":54,"Julia":53,"Katarina":52,"Kazumi":55,"Kazuya":54,"King":55,"Kuma":49,"Kunimitsu":0,"Lars":53,"Law":54,"Lee":55,"Lei":53,"Leo":53,"Leroy":55,"Lidia":46,"Lili":54,"LuckyChloe":54,"Marduk":53,"MasterRaven":53,"Miguel":53,"Negan":49,"Nina":52,"Noctis":54,"Panda":51,"Paul":50,"Shaheen":55,"Steve":56,"XiaoYu":55,"Yoshimitsu":56,"Zafina":54,"Omega":3.59,"youtube":"https://youtu.be/grazbUEJBLE","stlye":"Kunimitsu는 그녀의 어머니와 비슷하게 플레이하지만 두 개의 무기를 휘두르기 때문에 더 고급스럽습니다.그녀의 Rage Art는 그녀의 칼과 그녀의 kodachi가 이제 일반 불꽃과 푸른 불꽃으로 모두 박혀 있기 때문에 그녀의 kodachi로 빠르게 연속해서 상대방을 찌릅니다. 적에게 여러 번 베면 큰 피해를 입힙니다.","story":"어린 시절에 사소한 도둑이었던 Kunimitsu는 결국 악명 높은 Manji 일족에 들어가 속임수와 도둑 기술을 연마했습니다.그녀는 가문의 신조를 따라 부자에게서 빼앗아 가난한 사람들에게 나누어 주었지만, 결국 그녀는 가문에서 도둑질을 하고 자신에게 전리품을 보관하기 시작했습니다.이 행동은 클랜 지도자의 분노를 불러일으켰고 결국 그녀를 쫓아냈습니다."},
{"name":"Gigas","Rank":33,"Winrate":51.74,"Akuma":49,"Alisa":48,"Anna":47,"ArmorKing":52,"Asuka":54,"Bob":51,"Bryan":53,"Claudio":48,"DevilJin":53,"Dragunov":53,"Eddy":50,"Eliza":50,"Fahkumram":52,"Feng":54,"Ganryu":50,"Geese":48,"Gigas":0,"Heihachi":51,"Hwoarang":52,"Jack-7":51,"Jin":55,"Josie":53,"Julia":51,"Katarina":51,"Kazumi":54,"Kazuya":54,"King":53,"Kuma":52,"Kunimitsu":51,"Lars":51,"Law":53,"Lee":52,"Lei":53,"Leo":55,"Leroy":49,"Lidia":51,"Lili":52,"LuckyChloe":50,"Marduk":47,"MasterRaven":53,"Miguel":52,"Negan":54,"Nina":50,"Noctis":52,"Panda":54,"Paul":52,"Shaheen":52,"Steve":55,"XiaoYu":55,"Yoshimitsu":53,"Zafina":51,"Omega":1.41,"youtube":"https://youtu.be/DCBm3Ss24sQ","stlye":"기가스는 매우 직설적인 캐릭터입니다. 그의 기동성은 최악이고 그는 Tekken 7 명단의 다른 많은 구성원만큼 다재다능하거나 강력한 툴킷을 가지고 있지 않지만 그를 약한 저급 전투기로 치부해서는 안됩니다.","story":"한때는 평범한 인간이자 재능 있는 무술가였지만 이제는 투덜거리는 괴물로 전락했습니다. Gigas는 G Corporation이 궁극의 무기를 만들기 위해 주도한 잔인한 생명 공학 실험의 결과입니다."},
{"name":"Bob","Rank":34,"Winrate":50.5,"Akuma":51,"Alisa":48,"Anna":48,"ArmorKing":51,"Asuka":52,"Bob":0,"Bryan":52,"Claudio":49,"DevilJin":53,"Dragunov":52,"Eddy":51,"Eliza":49,"Fahkumram":48,"Feng":53,"Ganryu":45,"Geese":48,"Gigas":49,"Heihachi":53,"Hwoarang":52,"Jack-7":51,"Jin":54,"Josie":51,"Julia":50,"Katarina":49,"Kazumi":52,"Kazuya":53,"King":53,"Kuma":48,"Kunimitsu":46,"Lars":51,"Law":53,"Lee":53,"Lei":52,"Leo":51,"Leroy":45,"Lidia":46,"Lili":53,"LuckyChloe":48,"Marduk":49,"MasterRaven":54,"Miguel":52,"Negan":46,"Nina":52,"Noctis":50,"Panda":48,"Paul":52,"Shaheen":52,"Steve":53,"XiaoYu":54,"Yoshimitsu":56,"Zafina":47,"Omega":1.41,"youtube":"https://youtu.be/sFhpRtnjays","stlye":"밥은 그럼에도 불구하고 공격을 선호하는 만능 캐릭터입니다. 그의 툴킷은 광범위하고 균형이 잘 잡혀 있어 Bob은 유연하지만 공격적으로 플레이할 때 실제로 빛을 발하기 시작합니다.Bob의 포커 게임은 매우 강력하며 경기의 속도를 제어하고 상대방을 제압할 수 있습니다.그의 미시마 스타일 웨이브 대쉬는 다양한 믹스업으로 이어지며 내장된 움직임과 훌륭한 잽이 포함된 일부 원거리 공격과 함께 훌륭한 접근 도구 역할을 합니다.","story":"그의 핵심인 카리스마 있고 사랑스러운 캐릭터인 Robert Richards는 Tekken 시리즈의 풍자 만화가 특성을 구현합니다."},
{"name":"Geese","Rank":35,"Winrate":51.92,"Akuma":50,"Alisa":51,"Anna":49,"ArmorKing":50,"Asuka":53,"Bob":52,"Bryan":52,"Claudio":49,"DevilJin":53,"Dragunov":53,"Eddy":52,"Eliza":50,"Fahkumram":51,"Feng":52,"Ganryu":49,"Geese":0,"Gigas":52,"Heihachi":50,"Hwoarang":53,"Jack-7":51,"Jin":55,"Josie":56,"Julia":52,"Katarina":51,"Kazumi":56,"Kazuya":55,"King":52,"Kuma":51,"Kunimitsu":49,"Lars":53,"Law":53,"Lee":54,"Lei":53,"Leo":53,"Leroy":50,"Lidia":50,"Lili":51,"LuckyChloe":52,"Marduk":49,"MasterRaven":51,"Miguel":54,"Negan":54,"Nina":51,"Noctis":52,"Panda":52,"Paul":51,"Shaheen":53,"Steve":56,"XiaoYu":51,"Yoshimitsu":56,"Zafina":48,"Omega":2.05,"youtube":"https://youtu.be/g_DlpoMDh14","stlye":"2D 프랜차이즈에서 나왔음에도 불구하고 Geese는 Tekken의 시스템에 편안함을 느끼는 합리적으로 직선적인 캐릭터입니다. Akuma와 같은 사람과 달리 Geese는 미터에 그다지 의존하지 않습니다.","story":"범죄 군주이자 비즈니스 거물. 기스 하워드는 악마 유전자를 획득하여 더 큰 힘을 얻기 위해 King of Iron First 토너먼트에 합류합니다."},
{"name":"Leo","Rank":36,"Winrate":48.36,"Akuma":49,"Alisa":47,"Anna":45,"ArmorKing":49,"Asuka":51,"Bob":49,"Bryan":49,"Claudio":46,"DevilJin":53,"Dragunov":50,"Eddy":45,"Eliza":46,"Fahkumram":48,"Feng":49,"Ganryu":43,"Geese":47,"Gigas":45,"Heihachi":50,"Hwoarang":48,"Jack-7":47,"Jin":53,"Josie":49,"Julia":49,"Katarina":47,"Kazumi":51,"Kazuya":52,"King":50,"Kuma":47,"Kunimitsu":47,"Lars":48,"Law":50,"Lee":49,"Lei":49,"Leo":0,"Leroy":45,"Lidia":46,"Lili":48,"LuckyChloe":47,"Marduk":47,"MasterRaven":48,"Miguel":48,"Negan":47,"Nina":49,"Noctis":51,"Panda":46,"Paul":50,"Shaheen":50,"Steve":51,"XiaoYu":52,"Yoshimitsu":52,"Zafina":45,"Omega":1.03,"youtube":"https://youtu.be/n1SRmuVKkLU","stlye":"레오는 균형 잡힌 게임 플레이로 정의되며 거의 모든 것에 능숙합니다. 레오 클리젠은 빠르고 강력한 콤보와 다중 히트, 높고 낮은 타격을 주는 놀라운 타격을 많이 가지고 있습니다.레오는 움직임 사이에 지연이나 회복이 거의 없기 때문에 매우 안전합니다. 그러나 대부분의 공격은 매우 간단하고 예측 가능하며 우회하는 경향이 있습니다.","story":"Leo Kliesen은 어렸을 때 아버지를 잃었지만 여전히 용감하고 결단력 있는 사람으로 성장했습니다.그들의 어머니는 나중에 암살자에게 살해되었고, 그들은 이것을 미시마 카즈야로 의심했습니다. 레오는 모든 것을 알아내기 위해 King of Iron Fist Tournament에 참가하기로 결정합니다."},
{"name":"LuckyChloe","Rank":37,"Winrate":51.58,"Akuma":56,"Alisa":47,"Anna":52,"ArmorKing":50,"Asuka":54,"Bob":52,"Bryan":54,"Claudio":48,"DevilJin":54,"Dragunov":53,"Eddy":55,"Eliza":48,"Fahkumram":49,"Feng":52,"Ganryu":45,"Geese":48,"Gigas":50,"Heihachi":52,"Hwoarang":53,"Jack-7":52,"Jin":54,"Josie":53,"Julia":50,"Katarina":51,"Kazumi":52,"Kazuya":52,"King":54,"Kuma":52,"Kunimitsu":46,"Lars":51,"Law":55,"Lee":54,"Lei":57,"Leo":53,"Leroy":47,"Lidia":47,"Lili":51,"LuckyChloe":0,"Marduk":51,"MasterRaven":53,"Miguel":54,"Negan":49,"Nina":53,"Noctis":48,"Panda":51,"Paul":49,"Shaheen":53,"Steve":57,"XiaoYu":55,"Yoshimitsu":57,"Zafina":48,"Omega":1.28,"youtube":"https://youtu.be/lDPnAlNhD84","stlye":"LuckyChloe는 빠른 공격과 좋은 조합 게임을 가지고 있으며 그녀의 공격은 일반적으로 상당히 약하지만 큰 피해를 입힐 수 있는 좋은 콤보 옵션이 있습니다.그러나 그녀의 범위는 매우 짧고 카운터 옵션과 처벌자가 부족합니다. 즉, 그녀는 보다 공격적인 플레이 스타일을 선호하며 상대와 가까이 있고 지속적인 압박을 가합니다.","story":"LuckyChloe는 모델이 되고 싶었던 소녀입니다. 어느 날 도적이 그녀가 일하는 사무실에 왔습니다.그는 동료들을 때리기 시작했지만 그녀는 그의 길을 막았습니다.LuckyChloe는 그의 길을 막고 그녀가 그를 이기면 그가 댄스 그룹 멤버가 될 것이라고 내기를 제안했습니다. 그리고 행운의 소녀가 이겼습니다."},
{"name":"Yoshimitsu","Rank":38,"Winrate":45.8,"Akuma":45,"Alisa":41,"Anna":46,"ArmorKing":46,"Asuka":49,"Bob":44,"Bryan":47,"Claudio":41,"DevilJin":49,"Dragunov":48,"Eddy":42,"Eliza":44,"Fahkumram":45,"Feng":48,"Ganryu":45,"Geese":44,"Gigas":47,"Heihachi":47,"Hwoarang":45,"Jack-7":46,"Jin":50,"Josie":48,"Julia":46,"Katarina":45,"Kazumi":47,"Kazuya":49,"King":48,"Kuma":44,"Kunimitsu":44,"Lars":45,"Law":48,"Lee":47,"Lei":47,"Leo":48,"Leroy":46,"Lidia":43,"Lili":44,"LuckyChloe":43,"Marduk":46,"MasterRaven":47,"Miguel":46,"Negan":45,"Nina":45,"Noctis":44,"Panda":43,"Paul":45,"Shaheen":46,"Steve":50,"XiaoYu":48,"Yoshimitsu":0,"Zafina":44,"Omega":1.16,"youtube":"https://youtu.be/f0S1g9UE6P8","stlye":"요시미츠는 엄청나게 복잡한 캐릭터입니다. 닌자답게 그는 정통하지 않고 위험한 특성 때문에 잘 사용하려면 많은 경험과 연습이 필요한 모든 경우에 사용할 수 있는 광범위한 도구 모음을 가지고 있습니다.","story":"시시각각 변하는 모습을 지닌 만지족의 수수께끼의 지도자. 요시미츠는 도움이 필요한 사람들을 돕고 정의를 회복하기 위해 노력하는 고귀한 닌자입니다."},
{"name":"XiaoYu Ling","Rank":39,"Winrate":47.34,"Akuma":50,"Alisa":44,"Anna":47,"ArmorKing":48,"Asuka":50,"Bob":46,"Bryan":51,"Claudio":43,"DevilJin":52,"Dragunov":51,"Eddy":46,"Eliza":45,"Fahkumram":48,"Feng":46,"Ganryu":44,"Geese":49,"Gigas":45,"Heihachi":49,"Hwoarang":49,"Jack-7":47,"Jin":54,"Josie":49,"Julia":46,"Katarina":45,"Kazumi":49,"Kazuya":49,"King":51,"Kuma":40,"Kunimitsu":45,"Lars":47,"Law":50,"Lee":48,"Lei":51,"Leo":48,"Leroy":44,"Lidia":41,"Lili":47,"LuckyChloe":45,"Marduk":46,"MasterRaven":48,"Miguel":50,"Negan":44,"Nina":47,"Noctis":46,"Panda":43,"Paul":47,"Shaheen":47,"Steve":53,"XiaoYu":0,"Yoshimitsu":52,"Zafina":45,"Omega":1.67,"youtube":"https://youtu.be/IcTmSloiR8E","stlye":"Xiaoyu는 Yoshimitsu와 Lei Wulong 다음으로 까다로운 캐릭터일 것입니다. 그녀의 가장 명백한 장점 중 하나는 회피입니다.그녀의 Art of Phoenix 자세를 통해 그녀는 놀라운 수의 중간 공격을 피할 수 있을 만큼 충분히 낮아지고 많은 찌르기로 Xiaoyu가 아래로 내려가 높은 공격을 피할 수 있습니다.","story":"Ling Xiaoyu는 King of Iron First 토너먼트에 참가한 최연소 파이터 중 한 명입니다.그녀의 어린애 같은 태도와 천진한 외모에도 불구하고 그녀는 힘이 부족하지 않으며 세계에서 가장 강한 전사들과 맞붙을 수 있습니다."},
{"name":"Nina","Rank":40,"Winrate":50.46,"Akuma":51,"Alisa":49,"Anna":47,"ArmorKing":51,"Asuka":51,"Bob":48,"Bryan":52,"Claudio":47,"DevilJin":52,"Dragunov":52,"Eddy":49,"Eliza":51,"Fahkumram":52,"Feng":51,"Ganryu":45,"Geese":49,"Gigas":50,"Heihachi":52,"Hwoarang":52,"Jack-7":51,"Jin":54,"Josie":53,"Julia":50,"Katarina":48,"Kazumi":53,"Kazuya":53,"King":51,"Kuma":50,"Kunimitsu":48,"Lars":50,"Law":51,"Lee":52,"Lei":52,"Leo":51,"Leroy":47,"Lidia":47,"Lili":50,"LuckyChloe":47,"Marduk":51,"MasterRaven":53,"Miguel":51,"Negan":49,"Nina":0,"Noctis":51,"Panda":50,"Paul":49,"Shaheen":51,"Steve":54,"XiaoYu":53,"Yoshimitsu":55,"Zafina":47,"Omega":2.18,"youtube":"https://youtu.be/k7PbQISHyXo","stlye":"공격에 중점을 둔 키트를 사용하는 Nina는 상대를 돌진하는 데 특화되어 있는 매우 공격적인 캐릭터이며 공격의 맹공격을 만드는 다른 동작으로 찌르기를 취소하여 가차 없이 압박합니다.","story":"암살자 가문에서 태어난 Nina는 항상 목표물을 제거하는 데 집중하는 정신을 지닌 고도로 전문적이고 냉혈한 살인자입니다. 그녀의 감정 없는 태도에 균열을 일으킬 수 있는 것은 세상에 몇 가지뿐입니다."},
{"name":"Eddy","Rank":41,"Winrate":50.92,"Akuma":52,"Alisa":44,"Anna":50,"ArmorKing":50,"Asuka":54,"Bob":49,"Bryan":53,"Claudio":49,"DevilJin":54,"Dragunov":53,"Eddy":0,"Eliza":48,"Fahkumram":48,"Feng":51,"Ganryu":49,"Geese":48,"Gigas":50,"Heihachi":55,"Hwoarang":54,"Jack-7":51,"Jin":57,"Josie":51,"Julia":50,"Katarina":49,"Kazumi":54,"Kazuya":55,"King":54,"Kuma":49,"Kunimitsu":44,"Lars":49,"Law":50,"Lee":54,"Lei":53,"Leo":55,"Leroy":47,"Lidia":47,"Lili":49,"LuckyChloe":45,"Marduk":46,"MasterRaven":56,"Miguel":53,"Negan":48,"Nina":51,"Noctis":45,"Panda":49,"Paul":50,"Shaheen":55,"Steve":59,"XiaoYu":54,"Yoshimitsu":58,"Zafina":47,"Omega":0.9,"youtube":"https://youtu.be/TjTFUrZX0S8","stlye":"Eddy는 카포에라의 무술을 철저히 대표하는 완전히 독특한 움직임을 가진 매우 이례적인 캐릭터입니다.그의 사이드스텝은 매우 비효율적이지만 Eddy의 수평 이동은 최고 중 일부이므로 특히 무한 스테이지에서 장거리 이동을 잘 활용하고 적과의 공간을 확보할 수 있습니다.","story":" 고귀한 카포에라 마스터. 에디는 주변 사람들을 깊이 생각하고 미시마 카즈야를 죽이고 아버지의 복수를 하려는 남자입니다."},
{"name":"Anna","Rank":42,"Winrate":52.34,"Akuma":52,"Alisa":50,"Anna":0,"ArmorKing":53,"Asuka":53,"Bob":52,"Bryan":54,"Claudio":50,"DevilJin":54,"Dragunov":54,"Eddy":50,"Eliza":49,"Fahkumram":52,"Feng":51,"Ganryu":47,"Geese":51,"Gigas":53,"Heihachi":54,"Hwoarang":54,"Jack-7":49,"Jin":57,"Josie":56,"Julia":53,"Katarina":50,"Kazumi":57,"Kazuya":56,"King":54,"Kuma":46,"Kunimitsu":52,"Lars":53,"Law":54,"Lee":54,"Lei":53,"Leo":55,"Leroy":50,"Lidia":48,"Lili":52,"LuckyChloe":48,"Marduk":54,"MasterRaven":52,"Miguel":53,"Negan":52,"Nina":53,"Noctis":54,"Panda":51,"Paul":52,"Shaheen":52,"Steve":57,"XiaoYu":53,"Yoshimitsu":54,"Zafina":50,"Omega":1.54,"youtube":"https://youtu.be/KrkwoSUXMzs","stlye":"Anna는 매우 복잡한 캐릭터입니다. 그녀의 가장 뚜렷한 역학 중 하나는 혼돈의 심판 자세이며,믹스업과 강력한 움직임이 있는 공격적인 응용 프로그램뿐만 아니라 타이밍만 잘 맞으면 자동으로 던지기와 낮은 공격에 대응하므로 방어적인 응용 프로그램도 있습니다.","story":"팜므파탈의 정의. 안나의 섹시함은 무술 실력과 맞물려 부족할 땐 언제든 로켓 런처를 꺼낼 수 있다."},
{"name":"Lei","Rank":43,"Winrate":48.82,"Akuma":47,"Alisa":44,"Anna":47,"ArmorKing":50,"Asuka":49,"Bob":48,"Bryan":53,"Claudio":47,"DevilJin":52,"Dragunov":53,"Eddy":47,"Eliza":44,"Fahkumram":50,"Feng":47,"Ganryu":45,"Geese":47,"Gigas":47,"Heihachi":51,"Hwoarang":50,"Jack-7":50,"Jin":55,"Josie":51,"Julia":48,"Katarina":46,"Kazumi":51,"Kazuya":52,"King":52,"Kuma":45,"Kunimitsu":47,"Lars":51,"Law":52,"Lee":50,"Lei":0,"Leo":51,"Leroy":48,"Lidia":45,"Lili":46,"LuckyChloe":43,"Marduk":50,"MasterRaven":50,"Miguel":51,"Negan":47,"Nina":48,"Noctis":48,"Panda":44,"Paul":48,"Shaheen":51,"Steve":56,"XiaoYu":49,"Yoshimitsu":53,"Zafina":45,"Omega":1.41,"youtube":"https://youtu.be/zskWIaB59TY","stlye":"레이는 높은 위험도/높은 보상을 받는 캐릭터로, 상대를 헐뜯고 혼란스럽게 하기 위해 자세를 매우 훌륭하고 창의적으로 사용해야 합니다.","story":"Lei Wulong은 Tekken 7 세계에서 유명한 중국 배우 Jackie Chan의 원형입니다. 그는 범죄자들에게 진정한 위협이 되는 경찰관입니다. 조사 중 하나인 미시마 카즈야의 흔적을 쫓는다.그런데 갑자기 헤이하치가 그를 King of Iron Fist Tournament에 직접 초대합니다."},
{"name":"MasterRaven","Rank":44,"Winrate":49.06,"Akuma":50,"Alisa":44,"Anna":48,"ArmorKing":50,"Asuka":52,"Bob":46,"Bryan":51,"Claudio":48,"DevilJin":55,"Dragunov":52,"Eddy":44,"Eliza":47,"Fahkumram":47,"Feng":51,"Ganryu":46,"Geese":49,"Gigas":47,"Heihachi":52,"Hwoarang":50,"Jack-7":50,"Jin":55,"Josie":52,"Julia":50,"Katarina":47,"Kazumi":51,"Kazuya":55,"King":51,"Kuma":46,"Kunimitsu":47,"Lars":49,"Law":50,"Lee":51,"Lei":50,"Leo":52,"Leroy":45,"Lidia":44,"Lili":48,"LuckyChloe":47,"Marduk":47,"MasterRaven":0,"Miguel":50,"Negan":46,"Nina":47,"Noctis":46,"Panda":48,"Paul":48,"Shaheen":51,"Steve":53,"XiaoYu":52,"Yoshimitsu":53,"Zafina":47,"Omega":1.8,"youtube":"https://youtu.be/SM2NqkwFdrY","stlye":"MasterRaven은 견고하고 다재다능한 캐릭터이지만 그녀를 특별하게 만드는 몇 가지 단점이 있는 것은 아닙니다. 그녀의 광범위한 툴킷은 다양한 플레이 스타일로 이어집니다.수비 시 마스터 레이븐은 회피 동작과 강력한 역습 옵션을 활용하여 상대방을 궁지에 몰아넣을 수 있습니다.","story":"비밀에 싸인 닌자. 레이븐의 상위 선배인 마스터 레이븐은 자신의 임무를 달성하기 위해 속도, 힘, 인술 기술을 결합한 치명적인 전사입니다."},
{"name":"Akuma","Rank":45,"Winrate":49.08,"Akuma":0,"Alisa":47,"Anna":48,"ArmorKing":46,"Asuka":52,"Bob":49,"Bryan":50,"Claudio":48,"DevilJin":49,"Dragunov":48,"Eddy":48,"Eliza":52,"Fahkumram":44,"Feng":50,"Ganryu":49,"Geese":50,"Gigas":51,"Heihachi":49,"Hwoarang":51,"Jack-7":49,"Jin":52,"Josie":51,"Julia":48,"Katarina":47,"Kazumi":55,"Kazuya":48,"King":48,"Kuma":50,"Kunimitsu":45,"Lars":49,"Law":50,"Lee":51,"Lei":53,"Leo":51,"Leroy":45,"Lidia":46,"Lili":49,"LuckyChloe":44,"Marduk":46,"MasterRaven":50,"Miguel":47,"Negan":49,"Nina":49,"Noctis":45,"Panda":52,"Paul":48,"Shaheen":49,"Steve":54,"XiaoYu":50,"Yoshimitsu":55,"Zafina":48,"Omega":1.93,"youtube":"https://youtu.be/00HZEf2GYd4","stlye":"Street Fighter에서 Akuma는 강한 압력, 강력한 믹스업, 높은 데미지를 지닌 유리 대포이지만 이를 보상하기 위해 체력은 다소 낮습니다.","story":"권력에 집착하는 맹렬한 전사. Akuma는 강해지기 위해 무엇이든 멈추지 않는 무시무시한 전사입니다. 원래 프랜차이즈의 장벽을 허물기까지, Akuma는 Mishima Kazumi에게 빚을 갚기 위해 철권에 들어갑니다."},
{"name":"Kuma","Rank":46,"Winrate":51.88,"Akuma":50,"Alisa":45,"Anna":54,"ArmorKing":51,"Asuka":55,"Bob":52,"Bryan":54,"Claudio":49,"DevilJin":55,"Dragunov":53,"Eddy":51,"Eliza":51,"Fahkumram":46,"Feng":53,"Ganryu":51,"Geese":49,"Gigas":48,"Heihachi":54,"Hwoarang":51,"Jack-7":50,"Jin":56,"Josie":52,"Julia":53,"Katarina":50,"Kazumi":55,"Kazuya":57,"King":54,"Kuma":0,"Kunimitsu":51,"Lars":52,"Law":52,"Lee":53,"Lei":55,"Leo":53,"Leroy":47,"Lidia":47,"Lili":53,"LuckyChloe":48,"Marduk":49,"MasterRaven":54,"Miguel":50,"Negan":49,"Nina":50,"Noctis":50,"Panda":51,"Paul":54,"Shaheen":54,"Steve":56,"XiaoYu":60,"Yoshimitsu":56,"Zafina":51,"Omega":1.41,"youtube":"https://youtu.be/H92NCpNqtXI","stlye":"Panda 와 Kuma 는 동일한 움직임 세트와 프레임 데이터를 공유하므로 동일한 플레이 스타일을 공유합니다. 유일한 차이점은 Rage Art 애니메이션입니다. 곰은 흥미롭고 정통하지 않은 캐릭터입니다.","story":"미시마 헤이하치의 애완동물이자 가장 소중한 직원일 것입니다. 쿠마는 야생에서 볼 수 있는 어떤 곰보다 훨씬 똑똑하고 친근합니다.주인을 위해 싸우지 않을 때 그는 자신의 진정한 사랑인 판다의 애정을 얻기 위해 절망적으로 노력합니다."},
{"name":"Eliza","Rank":47,"Winrate":51.12,"Akuma":48,"Alisa":48,"Anna":51,"ArmorKing":50,"Asuka":53,"Bob":51,"Bryan":52,"Claudio":46,"DevilJin":54,"Dragunov":52,"Eddy":52,"Eliza":0,"Fahkumram":50,"Feng":54,"Ganryu":48,"Geese":50,"Gigas":50,"Heihachi":53,"Hwoarang":54,"Jack-7":51,"Jin":54,"Josie":53,"Julia":50,"Katarina":49,"Kazumi":54,"Kazuya":51,"King":52,"Kuma":49,"Kunimitsu":43,"Lars":51,"Law":52,"Lee":55,"Lei":56,"Leo":54,"Leroy":49,"Lidia":45,"Lili":51,"LuckyChloe":52,"Marduk":49,"MasterRaven":53,"Miguel":52,"Negan":49,"Nina":49,"Noctis":48,"Panda":49,"Paul":49,"Shaheen":54,"Steve":56,"XiaoYu":55,"Yoshimitsu":56,"Zafina":50,"Omega":1.54,"youtube":"https://youtu.be/pMMcRBMpQ3Q","stlye":" Eliza는 명백한 2D 캐릭터와 전통적인 Tekken 사이의 흥미로운 혼합입니다. 그녀는 두 범주 모두에서 탁월하지는 않지만 두 범주 모두에서 좋은 이점을 얻습니다.","story":"Eliza는 실수로 장기간 잠을 자다가 다시 힘을 되찾기 위해 노력하는 기면증 흡혈귀입니다. 그녀의 피에 대한 탐구는 끊임없는 낮잠으로 방해받지 않는다면 가차없고 끔찍할 것입니다."},
{"name":"Zafina","Rank":48,"Winrate":52.82,"Akuma":52,"Alisa":51,"Anna":50,"ArmorKing":55,"Asuka":55,"Bob":53,"Bryan":56,"Claudio":49,"DevilJin":56,"Dragunov":57,"Eddy":53,"Eliza":50,"Fahkumram":51,"Feng":51,"Ganryu":49,"Geese":52,"Gigas":49,"Heihachi":56,"Hwoarang":54,"Jack-7":53,"Jin":55,"Josie":55,"Julia":53,"Katarina":54,"Kazumi":54,"Kazuya":58,"King":56,"Kuma":49,"Kunimitsu":46,"Lars":53,"Law":55,"Lee":54,"Lei":55,"Leo":55,"Leroy":47,"Lidia":45,"Lili":53,"LuckyChloe":52,"Marduk":51,"MasterRaven":53,"Miguel":53,"Negan":50,"Nina":53,"Noctis":53,"Panda":50,"Paul":55,"Shaheen":54,"Steve":57,"XiaoYu":55,"Yoshimitsu":56,"Zafina":0,"Omega":1.67,"youtube":"https://youtu.be/w2levjTstcY","stlye":"","story":"비할 데 없는 부담을 지닌 강력한 파이터. Zafina는 고대 부족의 사명을 유지하고 악마 Azazel이 세상을 파괴하는 것을 막기 위해 싸웁니다."},
{"name":"Panda","Rank":49,"Winrate":51.36,"Akuma":48,"Alisa":48,"Anna":49,"ArmorKing":51,"Asuka":54,"Bob":52,"Bryan":54,"Claudio":48,"DevilJin":54,"Dragunov":51,"Eddy":51,"Eliza":51,"Fahkumram":46,"Feng":53,"Ganryu":48,"Geese":48,"Gigas":46,"Heihachi":54,"Hwoarang":51,"Jack-7":48,"Jin":57,"Josie":54,"Julia":54,"Katarina":49,"Kazumi":55,"Kazuya":56,"King":53,"Kuma":49,"Kunimitsu":49,"Lars":52,"Law":53,"Lee":53,"Lei":56,"Leo":54,"Leroy":50,"Lidia":45,"Lili":53,"LuckyChloe":49,"Marduk":51,"MasterRaven":52,"Miguel":50,"Negan":50,"Nina":50,"Noctis":49,"Panda":0,"Paul":53,"Shaheen":53,"Steve":56,"XiaoYu":57,"Yoshimitsu":57,"Zafina":50,"Omega":1.03,"youtube":"https://youtu.be/H92NCpNqtXI","stlye":"Panda 와 Kuma 는 동일한 움직임 세트와 프레임 데이터를 공유하므로 동일한 플레이 스타일을 공유합니다. 유일한 차이점은 Rage Art 애니메이션입니다. 곰은 흥미롭고 정통하지 않은 캐릭터입니다.Zafina는 훌륭하고 균형 잡힌 캐릭터입니다. 그녀의 뛰어난 움직임과 찌르기는 Zafina가 상대의 공간과 위치를 효율적으로 제어할 수 있게 해줍니다.","story":"미시마 헤이하치의 애완동물이자 가장 소중한 직원일 것입니다. 쿠마는 야생에서 볼 수 있는 어떤 곰보다 훨씬 똑똑하고 친근합니다.주인을 위해 싸우지 않을 때 그는 자신의 진정한 사랑인 판다의 애정을 얻기 위해 절망적으로 노력합니다."},
{"name":"Ganryu","Rank":50,"Winrate":53.10,"Akuma":51,"Alisa":51,"Anna":53,"ArmorKing":54,"Asuka":54,"Bob":55,"Bryan":54,"Claudio":54,"DevilJin":54,"Dragunov":55,"Eddy":51,"Eliza":52,"Fahkumram":48,"Feng":51,"Ganryu":0,"Geese":51,"Gigas":50,"Heihachi":53,"Hwoarang":54,"Jack-7":49,"Jin":57,"Josie":54,"Julia":54,"Katarina":52,"Kazumi":55,"Kazuya":56,"King":54,"Kuma":49,"Kunimitsu":49,"Lars":52,"Law":57,"Lee":53,"Lei":55,"Leo":57,"Leroy":47,"Lidia":50,"Lili":55,"LuckyChloe":55,"Marduk":52,"MasterRaven":54,"Miguel":54,"Negan":52,"Nina":55,"Noctis":54,"Panda":52,"Paul":55,"Shaheen":55,"Steve":56,"XiaoYu":56,"Yoshimitsu":55,"Zafina":51,"Omega":1.28,"youtube":"https://youtu.be/JiXIFqY6t2c","stlye":"한 기동성에도 불구하고, 간류는 상대방에게 집착하고 절대 놓지 않는 압박감 있는 캐릭터입니다.Ganryu는 압박이나 장난에서 벗어나기 어렵지만 일단 들어가면 위협적인 세력입니다.게임 최고의 저음 중 하나와 모호한 저/중 혼합으로 Ganryu는 상대방이 정확하게 추측하거나 빠져나오도록 압력의 틈을 찾도록 강요합니다.","story":"Ganryu는 다른 사람들의 필요와 프로젝트보다 자신의 이기적인 충동을 중시하는 탐욕스러운 스모 선수에게서 생각할 수 있습니다. 그리고 그것은 실제로 매우 가까운 설명이 될 것입니다. 그의 끊임없는 재정 문제의 나선형은 볼만한 광경입니다."},
{"name":"Lidia","Rank":51,"Winrate":54.10,"Akuma":54,"Alisa":53,"Anna":52,"ArmorKing":56,"Asuka":53,"Bob":54,"Bryan":51,"Claudio":55,"DevilJin":55,"Dragunov":54,"Eddy":53,"Eliza":55,"Fahkumram":53,"Feng":51,"Ganryu":50,"Geese":50,"Gigas":49,"Heihachi":54,"Hwoarang":55,"Jack-7":54,"Jin":55,"Josie":54,"Julia":56,"Katarina":53,"Kazumi":56,"Kazuya":55,"King":52,"Kuma":53,"Kunimitsu":54,"Lars":55,"Law":53,"Lee":53,"Lei":55,"Leo":54,"Leroy":55,"Lidia":0,"Lili":52,"LuckyChloe":53,"Marduk":56,"MasterRaven":56,"Miguel":58,"Negan":55,"Nina":53,"Noctis":57,"Panda":55,"Paul":53,"Shaheen":56,"Steve":56,"XiaoYu":59,"Yoshimitsu":57,"Zafina":55,"Omega":0,"youtube":"https://youtu.be/G39asfdMxSg","stlye":"리디아의 플레이 스타일은 양날의 검입니다. 그녀의 툴킷은 거의 전적으로 공격적인 플레이와 상대를 압박하는 데 중점을 두고 있습니다.그러나 그녀의 최선의 선택 중 많은 부분이 중단되거나 시작될 위험이 있습니다.따라서 Lidia는 접근 방식을 변경하거나 상대가 버튼을 존중하도록 조건을 변경해야 하며, 이는 다양한 피해 방지 옵션으로 수행할 수 있습니다.","story":"Lidia Sobieska가 폴란드의 최연소 총리가 된 이유를 쉽게 알 수 있습니다. 그녀의 애국심과 국민의 안녕을 위한 헌신은 누구에게도 뒤지지 않으며,심지어 조국을 위해 King of Iron First 토너먼트에 참가하기까지 합니다."}]

for i in ab:
    name = i['name']
    Rank = i['Rank']
    Winrate = i['Winrate']
    Akuma = i['Akuma']
    Alisa = i['Alisa']
    Anna = i['Anna']
    ArmorKing = i['ArmorKing']
    Asuka = i['Asuka']
    Bob = i['Bob']
    Bryan = i['Bryan']
    Claudio = i['Claudio']
    DevilJin = i['DevilJin']
    Dragunov = i['Dragunov']
    Eddy = i['Eddy']
    Eliza = i['Eliza']
    Fahkumram = i['Fahkumram']
    Feng = i['Feng']
    Ganryu = i['Ganryu']
    Geese = i['Geese']
    Gigas = i['Gigas']
    Heihachi = i['Heihachi']
    Hwoarang = i['Hwoarang']
    Jack = i['Jack-7']
    Jin = i['Jin']
    Josie = i['Josie']
    Julia = i['Julia']
    Katarina = i['Katarina']
    Kazumi = i['Kazumi']
    Kazuya = i['Kazuya']
    King = i['King']
    Kuma = i['Kuma']
    Kunimitsu = i['Kunimitsu']
    Lars = i['Lars']
    Law = i['Law']
    Lee = i['Lee']
    Lei = i['Lei']
    Leo = i['Leo']
    Leroy = i['Leroy']
    Lidia = i['Lidia']
    Lili = i['Lili']
    LuckyChloe = i['LuckyChloe']
    Marduk = i['Marduk']
    MasterRaven = i['MasterRaven']
    Miguel = i['Miguel']
    Negan = i['Negan']
    Nina = i['Nina']
    Noctis = i['Noctis']
    Panda = i['Panda']
    Paul = i['Paul']
    Shaheen = i['Shaheen']
    Steve= i['Steve']
    XiaoYu = i['XiaoYu']
    Yoshimitsu = i['Yoshimitsu']
    Zafina = i['Zafina']
    Omega = i['Omega']
    youtube = i['youtube']
    stlye = i['stlye']
    story = i['story']

    print(name)

Paul Phoenix
Kazuya
Bryan
Fahkumram
Leroy
ArmorKing
Dragunov
DevilJin
Jin
King
Hwaorang
Negan
Heihachi
SteveFox
Kazumi
Law
Lars
Claudio
Josie
Noctis
Miguel
Shaheen
Marduk
Asuka
Julia
Feng
Lili
Lee
Jack-7
Alisa
Katarina
Kunimitsu
Gigas
Bob
Geese
Leo
LuckyChloe
Yoshimitsu
XiaoYu Ling
Nina
Eddy
Anna
Lei
MasterRaven
Akuma
Kuma
Eliza
Zafina
Panda
Ganryu
Lidia


In [39]:
bp = Blueprint("char_detail", __name__, url_prefix="/char_detail",
               static_folder="static", template_folder="templates")


@bp.route('/<name>')
def home(name):

    char = db.char.find_one({'name': name}, {'_id': False})
    name = char['name']
    Rank = char['Rank']
    Winrate = char['Winrate']
    Omega = char['Omega']
    style = char['stlye']
    story = char['story']
    enemy = dict(itertools.islice(char.items(), 3, 54))
    cd = enemy.keys())
    vs_enemy = []
    
  
    for i in cd:
        if i != name:
            print(cd)
            # vs_enemy.append(i)
print(cd) 

IndentationError: unexpected indent (3472213271.py, line 23)